In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow.keras.backend as K
import pandas as pd
import numpy as np
import pickle as pkl

In [2]:
data_path = 'C:/Users/40107904/OneDrive - Anheuser-Busch InBev/ABI/WORK/hackathon_power/hackathon_lt_equity/dummy_data/processed_data/preprocessed_data.pkl'

with open(data_path, 'rb') as f:
    data_dict = pkl.load(f)

print(data_dict.keys())

dict_keys(['input_data', 'output_data'])


# Data processing

In [3]:
# train ds
num_control_features = data_dict["input_data"]["controls"].shape[1]

inputs_dict = {
    "time_idx": data_dict["input_data"]["time_idx"],
    "month_sin": data_dict["input_data"]["month_sin"],
    "month_cos": data_dict["input_data"]["month_cos"],
    "country_id": data_dict["input_data"]["country_id"],
    "brand_id": data_dict["input_data"]["brand_id"],
    "controls": data_dict["input_data"]["controls"],
}

targets = (data_dict["input_data"]["y_true"]/100, data_dict["input_data"]["group_id"])

BATCH_SIZE = 15
train_ds = tf.data.Dataset.from_tensor_slices((inputs_dict, targets))
train_ds = train_ds.batch(BATCH_SIZE) 

In [4]:
# predict_x
predict_x = {
    "time_idx": data_dict["output_data"]["time_idx"],
    "month_sin": data_dict["output_data"]["month_sin"],
    "month_cos": data_dict["output_data"]["month_cos"],
    "country_id": data_dict["output_data"]["country_id"],
    "brand_id": data_dict["output_data"]["brand_id"],
    "controls": data_dict["output_data"]["controls"],
}

test_targets = (data_dict["output_data"]["y_true"]/100, data_dict["output_data"]["group_id"])

# Model Arch

In [5]:

# -----------------------
# Model hyperparameters
# -----------------------
COUNTRY_VOCAB = 2      # set >= actual distinct countries
BRAND_VOCAB = 10       # set >= actual distinct brands
COUNTRY_EMB = 4
BRAND_EMB = 10
TIME_HIDDEN = 128
CAT_HIDDEN = 32
CONTROL_HIDDEN = 256
CONTROL_L2 = 1e-3       # regularize controllable contribution
DROPOUT = 0.1

# -----------------------
# Inputs spec (per-row)
# -----------------------
# Numeric/time
time_idx_in = layers.Input(shape=(1,), dtype="int32", name="time_idx")        # monotonic month index
month_sin_in = layers.Input(shape=(1,), dtype="float32", name="month_sin")
month_cos_in = layers.Input(shape=(1,), dtype="float32", name="month_cos")

# Categorical (to embedding)
country_in = layers.Input(shape=(1,), dtype="int32", name="country_id")
brand_in = layers.Input(shape=(1,), dtype="int32", name="brand_id")

# Controllable continuous features (vector). During real forecasting these will be NaN or masked.
controls_in = layers.Input(shape=(num_control_features,), dtype="float32", name="controls")  
# NOTE: replace None with exact number of control features (e.g. 10) in practice.

# Group id for per-(Country,date) grouping (int). Required for custom loss/metrics.
group_id_in = layers.Input(shape=(1,), dtype="int32", name="group_id")

# -----------------------
# Subnet: time/baseline f_time
# -----------------------
# Convert time index to float and optionally an embedding
time_float = layers.Lambda(lambda x: tf.cast(x, tf.float32))(time_idx_in)
t = layers.Concatenate()([time_float, month_sin_in, month_cos_in])
t = layers.Dense(TIME_HIDDEN, activation="relu")(t)
t = layers.Dropout(DROPOUT)(t)
t = layers.Dense(TIME_HIDDEN//2, activation="relu")(t)
# Single scalar baseline contribution per row
time_out = layers.Dense(1, activation=None, name="time_out")(t)   # no activation: logit-space contribution

# -----------------------
# Subnet: country effect f_country
# -----------------------
c_emb = layers.Embedding(input_dim=COUNTRY_VOCAB, output_dim=COUNTRY_EMB, name="country_emb")(country_in)
c = layers.Flatten()(c_emb)
c = layers.Dense(CAT_HIDDEN, activation="relu")(c)
country_out = layers.Dense(1, activation=None, name="country_out")(c)

# -----------------------
# Subnet: brand effect f_brand
# -----------------------
b_emb = layers.Embedding(input_dim=BRAND_VOCAB, output_dim=BRAND_EMB, name="brand_emb")(brand_in)
b = layers.Flatten()(b_emb)
b = layers.Dense(CAT_HIDDEN, activation="relu")(b)
brand_out = layers.Dense(1, activation=None, name="brand_out")(b)

# -----------------------
# Subnet: controllables f_controls
# -----------------------
# We'll allow the model to accept a vector of controllable features.
# To enable training with missing controllables (forecasting), you'll feed zeros and a binary mask as part of the inputs,
# or use a dataset map that randomly zeroes controls during training (see dataset example below).
ctrl = layers.Dense(CONTROL_HIDDEN, activation="relu")(controls_in)
ctrl = layers.Dropout(DROPOUT)(ctrl)
ctrl = layers.Dense(CONTROL_HIDDEN//2, activation="relu")(ctrl)
# single scalar small contribution
ctrl_out_raw = layers.Dense(1, activation=None, name="ctrl_out_raw")(ctrl)

# apply a small learned scaling to controllables (keeps them small)
# scale parameter constrained positive; initialized small
ctrl_scale = tf.Variable(initial_value=0.01, dtype=tf.float32, trainable=True, name="ctrl_scale")
ctrl_out = layers.Lambda(lambda x: x * ctrl_scale, name="ctrl_out")(ctrl_out_raw)

bias_in = layers.Input(shape=(1,), dtype="float32", name="bias_in")
bias_out = layers.Dense(1, use_bias=True, kernel_initializer="zeros")(bias_in)

# -----------------------
# Baseline (time + country + brand)
# -----------------------
baseline = layers.Add(name="baseline")([time_out, country_out, brand_out])  # (batch, 1)

# -----------------------
# Variability (controls, downweighted ~10%)
# -----------------------
variability = layers.Lambda(lambda x: 0.1 * x, name="variability")(ctrl_out)

# -----------------------
# Combine
# -----------------------
add_logit = layers.Add(name="add_logit")([baseline, variability])  # (batch, 1)


# Model returns per-row logit and group_id (group_id is passed through for loss)
model = keras.Model(
    inputs=[time_idx_in, month_sin_in, month_cos_in, country_in, brand_in, controls_in],
    outputs=[add_logit, ctrl_out],  # return ctrl_out so we can regularize in loss if desired
    name="gam_additive_power_model"
)

# -----------------------
# Custom loss: segment-wise softmax + MSE on percentages
# -----------------------
# We expect y_true to be `power_pct` (e.g. 0-100) per row in the same order as logits.
# group_ids must be integers that are identical for rows belonging to the same Country x Date group.

# Custom training step via custom loss function wrapper
# Keras expects loss(y_true, y_pred); we'll wrap to accept logits and group_ids passed as model outputs.
# Our model returns [logits, group_id, ctrl_out]. For training, we'll pass y_true (power_pct) only.
# We'll create a custom loss object that extracts logits and group_ids from y_pred tuple.

def fractions_to_logits_tf(y_true, group_ids, eps=1e-8):
    """
    Convert true fractions (per group) to centered logits.
    Args:
        y_true:    (N,) float32 tensor, fractions per row (sum=1 per group)
        group_ids: (N,) int32 tensor, group index per row
        eps:       small constant to avoid log(0)
    Returns:
        logits: (N,) float32 tensor
    """
    y_true = tf.reshape(y_true, [-1])
    group_ids = tf.reshape(group_ids, [-1])

    # clip to avoid log(0)
    y_true = tf.clip_by_value(y_true, eps, 1.0)

    # raw logits = log(p)
    raw_logits = tf.math.log(y_true)

    # subtract group mean so logits are centered (softmax is shift-invariant)
    num_groups = tf.reduce_max(group_ids) + 1
    group_means = tf.math.unsorted_segment_mean(raw_logits, group_ids, num_groups)
    centered_logits = raw_logits - tf.gather(group_means, group_ids)

    return centered_logits



def grouped_loss(y_true, logits, group_ids, ctrl_out, num_brands, ctrl_reg=1e-3, smooth=0.05, logit_reg=1e-4):
    """
    Grouped softmax loss with:
      - label smoothing to avoid 0/100 collapse
      - logit penalty to prevent extreme values
      - control penalty to keep controls small

    y_true:   (N,) float32, original KPI values (e.g. 0–100 scale)
    logits:   (N,) float32, model outputs before softmax
    group_ids:(N,) int32, group index for each row
    ctrl_out: (N,1) float32, control contribution
    num_brands: int, number of brands per group
    """

    y_true = tf.reshape(y_true, [-1])
    logits = tf.reshape(logits, [-1])
    group_ids = tf.reshape(group_ids, [-1])

    num_groups = tf.reduce_max(group_ids) + 1
    eps = 0

    # normalize y_true within each group → fractions
    group_sum_true = tf.math.unsorted_segment_sum(y_true, group_ids, num_groups)
    true_frac = y_true / (tf.gather(group_sum_true, group_ids) + eps)

    # label smoothing
    true_frac = true_frac * (1.0 - smooth) + smooth / float(num_brands)

    # predicted fractions (per-group softmax)
    logits = tf.clip_by_value(logits, -20.0, 20.0)  # stability
    exp_logits = tf.exp(logits - tf.reduce_max(logits))
    seg_sum = tf.math.unsorted_segment_sum(exp_logits, group_ids, num_groups)
    pred_frac = exp_logits / (tf.gather(seg_sum, group_ids) + eps)

    # mean squared error on fractions
    # mse = tf.reduce_mean(tf.square(pred_frac - true_frac))
    mse = tf.reduce_mean(tf.square(logits - fractions_to_logits_tf(y_true, group_ids)))

    # penalties
    ctrl_pen = tf.reduce_mean(tf.square(ctrl_out)) * ctrl_reg
    logit_pen = tf.reduce_mean(tf.square(logits)) * logit_reg

    return 100 * mse + ctrl_pen + logit_pen


# -----------------------
# compile model
# -----------------------

model.summary()

# -----------------------
# Recommended: custom training loop (fit with tf.data and GradientTape)
# -----------------------
# Outline of what training loop should do:
# - dataset yields (feature_dict, y_true)
# - feature_dict contains 'group_id' as an input
# - inside loop, call model to get [logits, group_id_out, ctrl_out]
# - compute loss with GroupedMSELoss on logits/group_id and y_true
# - backward, optimizer.step

# Example pseudo-code for training loop:

# -----------------------
# Dataset preparation notes (important)
# -----------------------
# - You must prepare `group_id` such that all rows for the same Country x date share exactly the same integer id.
#   Example: group_id = country_enc * 100000 + time_idx  (ensure unique across countries & time range)
# - Build tf.data so each batch contains a mix of complete groups. For stable training, ensure groups are not split
#   across batches mid-group. The simplest approach is to batch by groups (pad groups to max rows per group per batch).
# - Alternatively, you can rely on tf.math.segment_* which tolerates out-of-order segment ids but NEEDS group_id contiguous
#   within the batch or requires correct segment_ids and segment_max. Practically, produce a dataset where each element
#   is a whole group (Country x date) with variable #brands (but here brands per group is constant), then flatten.

# -----------------------
# Output notes
# -----------------------
# - At inference, the model will produce per-row logits. Apply the same segment_softmax using group_ids to get predicted fractions.
# - Multiply fractions by 100 -> predicted percent shares that sum to 100 per Country x date.

# -----------------------
# End of model definition
# -----------------------


Model: "gam_additive_power_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ time_idx            │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 1)         │          0 │ time_idx[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ month_sin           │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ month_cos           │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ controls            │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 3)         │          0 │ lambda[0][0],     │
│ (Concatenate)       │                   │            │ month_sin[0][0],  │
│                     │                   │            │ month_cos[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ country_id          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ brand_id            │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256)       │      3,072 │ controls[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │        512 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ country_emb         │ (None, 1, 4)      │          8 │ country_id[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ brand_emb           │ (None, 1, 10)     │        100 │ brand_id[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 4)         │          0 │ country_emb[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 10)        │          0 │ brand_emb[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │     32,896 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │        160 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │        352 │ flatten_1[0][0] 

 Total params: 45,616 (178.19 KB)

 Trainable params: 45,616 (178.19 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# train
optimizer = keras.optimizers.Adam(learning_rate=0.01)

@tf.function
def train_step(batch_x, y_true, group_ids):
    with tf.GradientTape() as tape:
        logits, ctrl_out = model(batch_x, training=True)
        # loss_value = grouped_loss_old(y_true, logits, group_ids, ctrl_out, ctrl_reg=1e-3)
        # loss_value = simple_loss(y_true, logits, ctrl_out, ctrl_reg=1e-1)
        # loss_value = mse_loss(y_true, logits)
        loss_value = grouped_loss(
            y_true=y_true,
            logits=logits,
            group_ids=group_ids,
            ctrl_out=ctrl_out,
            num_brands=5,
            ctrl_reg=1e-3,
            smooth=0.005,
            logit_reg=1e-4
        )
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss_value


for epoch in range(100):
    epoch_losses = []
    for (batch_x, (y_true, group_ids)) in train_ds:  # your tf.data must yield this structure
        loss_value = train_step(batch_x, y_true, group_ids)
        epoch_losses.append(loss_value.numpy())
    print(f"Epoch {epoch+1}: loss = {np.mean(epoch_losses):.4f}", flush=True)

Epoch 1: loss = 94.7715
Epoch 2: loss = 26.2535
Epoch 3: loss = 11.2410
Epoch 4: loss = 15.1532
Epoch 5: loss = 19.3892
Epoch 6: loss = 5.6420
Epoch 7: loss = 3.2072
Epoch 8: loss = 2.4359
Epoch 9: loss = 1.8711
Epoch 10: loss = 2.0367
Epoch 11: loss = 1.9289
Epoch 12: loss = 2.0391
Epoch 13: loss = 1.9634
Epoch 14: loss = 2.0735
Epoch 15: loss = 1.9790
Epoch 16: loss = 1.9136
Epoch 17: loss = 1.8213
Epoch 18: loss = 1.8339
Epoch 19: loss = 1.6644
Epoch 20: loss = 1.5956
Epoch 21: loss = 1.4497
Epoch 22: loss = 1.3313
Epoch 23: loss = 1.4484
Epoch 24: loss = 1.3445
Epoch 25: loss = 1.1843
Epoch 26: loss = 1.1416
Epoch 27: loss = 1.4381
Epoch 28: loss = 2.3359
Epoch 29: loss = 1.9400
Epoch 30: loss = 1.3363
Epoch 31: loss = 1.2400
Epoch 32: loss = 1.1295
Epoch 33: loss = 1.0596
Epoch 34: loss = 0.9655
Epoch 35: loss = 1.0267
Epoch 36: loss = 0.9332
Epoch 37: loss = 1.0144
Epoch 38: loss = 0.8359
Epoch 39: loss = 0.8698
Epoch 40: loss = 0.9966
Epoch 41: loss = 1.0216
Epoch 42: loss = 0.9

In [7]:
# predict
logits, ctrl_out = model(predict_x, training=False)
logits = logits.numpy().reshape(-1)
print(logits)

[ 1.0377107   0.20347999 -0.27686808 -0.70040536 -0.24462394  1.0515219
  0.21292531 -0.28903604 -0.7122316  -0.31738013  1.0240077   0.20376076
 -0.29924533 -0.7154948  -0.31299892]


In [35]:
def logits_to_percentages(logits, group_ids):
    """
    Convert logits -> per-group percentages (summing to 100).
    
    logits:    (N,) array of raw model outputs
    group_ids: (N,) array of group IDs (int)
    """
    logits = np.asarray(logits).reshape(-1)
    group_ids = np.asarray(group_ids).reshape(-1)

    softmax = np.zeros_like(logits, dtype=float)

    for gid in np.unique(group_ids):
        mask = (group_ids == gid)
        group_logits = logits[mask]

        # stable softmax within group
        exp_logits = np.exp(group_logits - np.max(group_logits))
        softmax[mask] = exp_logits / exp_logits.sum()



    return (softmax * 100)  # percentages, rounded to 1 decimal


pred_pct = logits_to_percentages(logits, data_dict["output_data"]["group_id"])

In [36]:
num_brands = data_dict["output_data"]["brand_id"].max() + 1

In [37]:
pred_pct = pred_pct.reshape(-1,num_brands)
for i in range(len(pred_pct)):
    pred_pct[i] = pred_pct[i] / pred_pct[i].sum() * 100

pred_pct = pred_pct.reshape(-1)

In [38]:
# model predicted powers
print(pred_pct.reshape(-1,num_brands))  

[[46.38141096 20.13919801 12.45746464  8.15623552 12.86569089]
 [47.17419744 20.3941986  12.345469    8.08567256 12.00046241]
 [46.62376275 20.52951127 12.41440158  8.1874943  12.2448301 ]]


In [39]:
print(pred_pct.reshape(-1,num_brands).sum(axis=1))  # should be all 100s

[100. 100. 100.]


In [40]:
# true powers
y_pred_true = data_dict["output_data"]["y_true"]
print(y_pred_true.reshape(-1,num_brands))

[[46.9 19.2 13.3  9.1 11.5]
 [46.9 19.2 13.3  9.1 11.5]
 [46.4 19.  13.5  9.3 11.8]]


In [41]:
# rmse skill
rmse = np.sqrt(np.mean((pred_pct - y_pred_true)**2))
print(f"Test RMSE: {rmse:.4f}")
standard_deviation = np.std(y_pred_true)
print(f"Standard Deviation of true values: {standard_deviation:.4f}")
rmse_skill = max(0, min(1, 1-(rmse/standard_deviation)))
print(rmse_skill)

Test RMSE: 0.9421
Standard Deviation of true values: 13.7655
0.9315641964624685


In [44]:
# trend hit by brand
num_brands = data_dict["output_data"]["brand_id"].max() + 1

trend_hits = 0
total_trends = (len(y_pred_true) / num_brands - 1 ) * num_brands

reshaped_pred = pred_pct.reshape(-1, num_brands)
reshaped_true = y_pred_true.reshape(-1, num_brands)

reshaped_pred_diff = np.diff(reshaped_pred, axis=0)
reshaped_true_diff = np.diff(reshaped_true, axis=0)

reshaped_pred_sign = np.sign(reshaped_pred_diff)
reshaped_true_sign = np.sign(reshaped_true_diff)

trend_hits = np.sum(reshaped_pred_sign == reshaped_true_sign)

trend_hit_rate = trend_hits / total_trends
print(f"Total Trends: {total_trends}")
print(f"Trend Hits: {trend_hits}")
print(f"Trend Hit Rate: {trend_hit_rate:.4f}")

Total Trends: 10.0
Trend Hits: 4
Trend Hit Rate: 0.4000


In [43]:
# final score
final_score = 0.5 * (rmse_skill + trend_hit_rate)
print(f"Final Score: {final_score:.4f}")

Final Score: 0.6658
